In [1]:
from bs4 import BeautifulSoup
import requests
from contextlib import closing
from selenium.webdriver import Firefox
import time
from gpx_converter import Converter
import random
import os
import json

In [11]:
start_distance = 30
length_min = 15
length_max = 100
for location in ['Albuquerque', 'Salt Lake City']:
    path = 'hub_gpx/'+location.lower().replace(' ','_')+'/'
    os.makedirs(path, exist_ok=True)
    tmp_route_file = path+'routes.json'
    loc_url = location.lower().replace(' ','+')
    route_list = []
    for offset in [0,20,40,60]:
        with closing(Firefox()) as browser:
            browser.get(f"https://ridewithgps.com/find#search/0/search%5Boffset%5D={offset}&search%5Bdedupe%5D=1&search%5Bstart_distance%5D={start_distance}&search%5Bstart_location%5D={loc_url}&search%5Bkeywords%5D=&search%5Blength_min%5D={length_min}&search%5Blength_max%5D={length_max}&search%5Belevation_min%5D=0&search%5Belevation_max%5D=10000&search%5Broute_type%5D%5Bpoint_to_point%5D=1&search%5Bsort_by%5D=")
            time.sleep(5)
            page_source = browser.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        tst = soup.body.find_all('div',class_='search_result')
        for t in tst:
            if 'routes' in t.h2.a['href']:
                name = t.h2.a.text.strip()
                url = f"https://ridewithgps.com{t.h2.a['href']}"

                img = []
                if t.find(class_='imagecells') and t.find(class_='imagecells').img:
                    img.append(t.find(class_='imagecells').img['src'])
                if t.find(class_='static-map') and t.find(class_='static-map').img:
                    img.append('https://ridewithgps.com/'+t.find(class_='static-map').img['src'])
                if t.find(class_='photos_viewer') and t.find(class_='photos_viewer').img:
                    img.append('https://ridewithgps.com/'+t.find(class_='photos_viewer').img['src'])


                desc = []
                if t.find('p',class_='route-overview') and t.find('p',class_='route-overview').text:
                    desc.append(t.find('p',class_='route-overview').text.strip())
                if t.find('p',class_='description_and_byline') and t.find('p',class_='description_and_byline').text:
                    desc.append(t.find('p',class_='description_and_byline').text.strip())

                gpx_page = requests.get(url+'.gpx?sub_format=track')
                if gpx_page.status_code==403:
                    print('ruh-roh')
                    time.sleep(10)
                    continue
                gpx_file = path+url.split('/')[-1]+".gpx"
                with open(gpx_file, "wb") as outfile:
                    outfile.write(gpx_page.content)
                try:
                    dic = Converter(input_file=gpx_file).gpx_to_dictionary()

                    route_list.append({'url':url,
                                       'name': name,
                                       'description': desc,
                                       'images': img,
                                       'coord': [[dic['longitude'][i],dic['latitude'][i]]for i in range(len(dic['latitude']))]})

                    with open(tmp_route_file, "w") as outfile:
                        json.dump(route_list, outfile, indent=4)
                except:
                    print('something broke')

                time.sleep(5+random.random()*30)
        time.sleep(5+random.random()*10)
    time.sleep(5+random.random()*10)

something broke
something broke
something broke
something broke


In [9]:
location

'Minneapolis'